<a href="https://colab.research.google.com/github/Soumi7/EY_GDS_Project/blob/main/EY_GDS_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/MyDrive/EY_DATA/saved_model/ey_model2')

In [4]:
!pip install bert-for-tf2

In [5]:
!pip install -r /content/drive/MyDrive/EY_DATA/requirements.txt

In [6]:
import bert
from bert.tokenization.bert_tokenization import FullTokenizer
import numpy as np

In [7]:
classes = ['Research', 'Coding Guidelines', 'Case Study', 'Financial Reports', 'CompanyDetails', 'AuditProposals']

In [ ]:
import os
#import PyPDF2
#import data_func
import csv
import pandas as pd
from io import StringIO
import re
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from nltk.tokenize import sent_tokenize
import csv
import nltk
nltk.download('punkt')
import sklearn
import numpy as np
from sklearn.utils import shuffle

def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            os.path.join(root, name)
            r.append(os.path.join(root, name))
            

    return r

dir="/content/drive/My Drive/EY_DATA"

csv_data=[]
file_loc=[]
intent=[]

label=[]
file_names=[]
file_name=[]

for filename in list_files(dir):
    #printing file names
    print(filename)
    sentence =[]
    if filename.endswith('.pdf'):
        output_string = StringIO()
        with open(filename, 'rb') as in_file:
            parser = PDFParser(in_file)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = TextConverter(rsrcmgr, output_string, laparams = LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)
        # print(type(output_string.getvalue()))
        # print(type(output_string))
        text=output_string.getvalue()
        data = sent_tokenize(text)
        for line in data:
            res=re.sub('\s+',' ',line)
            line=str(res)
            sentence.append(line)            
    
        #here we load trained model and get predictions on each sentence in test_data_from_crawler_pdf
        #y_pred = model.predict(data.test_x).argmax(axis=-1)

    #sentences = ["we are studying for exam"]

    tokenizer = FullTokenizer(vocab_file= "/content/drive/MyDrive/EY_DATA/vocab.txt")
    
    pred_tokens = map(tokenizer.tokenize, sentence)
    pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
    pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))

    pred_token_ids = map(lambda tids: tids +[0]*(128-len(tids)),pred_token_ids)
    pred_token_ids = np.array(list(pred_token_ids))

    #################################################################
    #                 Taking predictions from model                 #
    #################################################################

    predictions = new_model.predict(pred_token_ids).argmax(axis=-1)

    intents_we_came_across = []
    #print(filename)

    for text, label in zip(sentence, predictions):
        print("text:", text, "\nintent:", classes[label])
        if classes[label] not in intents_we_came_across:
            intents_we_came_across.append(classes[label])
        print()

    for count in range(len(intents_we_came_across)):
        file_loc.append(filename)
        file_name.append(filename.split("/")[-1])
        intent.append(intents_we_came_across[count])
    

df = pd.DataFrame(list(zip(file_loc, file_name, intent)) , columns=["File Location", "File Name", "Intent"])
df.to_csv('test_data_from_crawler.csv',encoding='utf-8-sig', index=False)

In [ ]:
while True:
    n = input("Enter query")
    if n == "EXIT":
        print("Exiting")
        break
    # m = input("Enter 2nd query")
    sentence = [n]

    tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/EY_DATA/vocab.txt")

    pred_tokens = map(tokenizer.tokenize, sentence)
    pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
    pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
    
    max_seq_len=128

    pred_token_ids = map(lambda tids: tids +[0]*(max_seq_len-len(tids)),pred_token_ids)
    pred_token_ids = np.array(list(pred_token_ids))

    # print(type(pred_token_ids))

    

    predictions = new_model.predict(pred_token_ids).argmax(axis=-1)

    intents_we_came_across = []
    for text, label in zip(sentence, predictions):
        # print("text:", text, "\nintent:", classes[label])
        intents_we_came_across.append(classes[label])
        print()

    for count in range(len(intents_we_came_across)):
        file_loc.append(filename)
        file_name.append(filename.split("/")[-1])
        intent.append(intents_we_came_across[count])
    
    # print(intent)

    df = pd.read_csv("/content/folders.csv", encoding = "utf-8-sig")
    # df = df.drop_duplicates(subset ="File Name") 
    # df.head()
    for word in intents_we_came_across:
        for i in range(len(df['Intent'])):
            if df['Intent'][i]==word:
                print(df["File Name"][i])
                print(df["File Location"][i])
                print()

Streaming output truncated to the last 5000 lines.
/home/lohith/Desktop/EY Hackathon/EY_GDS_Project/data_to_preprocess/AuditProposals/Energy_audit_proposal_for_industry_applied_to_a_ca.pdf

Energy_audit_proposal_for_industry_applied_to_a_ca.pdf
/home/lohith/Desktop/EY Hackathon/EY_GDS_Project/data_to_preprocess/AuditProposals/Energy_audit_proposal_for_industry_applied_to_a_ca.pdf

Energy_audit_proposal_for_industry_applied_to_a_ca.pdf
/home/lohith/Desktop/EY Hackathon/EY_GDS_Project/data_to_preprocess/AuditProposals/Energy_audit_proposal_for_industry_applied_to_a_ca.pdf

Energy_audit_proposal_for_industry_applied_to_a_ca.pdf
/home/lohith/Desktop/EY Hackathon/EY_GDS_Project/data_to_preprocess/AuditProposals/Energy_audit_proposal_for_industry_applied_to_a_ca.pdf

Energy_audit_proposal_for_industry_applied_to_a_ca.pdf
/home/lohith/Desktop/EY Hackathon/EY_GDS_Project/data_to_preprocess/AuditProposals/Energy_audit_proposal_for_industry_applied_to_a_ca.pdf

Energy_audit_proposal_for_industry